In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fastparquet

In [ ]:
df = pd.read_parquet("meow_df.parquet", engine = 'fastparquet')

df.columns

df['time_diff'] = df.seconds_in_bucket - df.seconds_in_bucket_before

df = df.query("""time_id == time_id_before and stock_id == stock_id_before""")

df = df.drop(columns = ['time_id_before', 'seconds_in_bucket_before', 'stock_id_before', 'level_0'])

df['trade'] = (df.price >= df.ask_price1_before).astype(int) - (df.price <= df.bid_price1_before).astype(int)

In [ ]:
df['mid'] = (df.ask_price1 + df.bid_price1)/2

df['spread'] = (df.ask_price1 - df.bid_price1)*10000/df.mid

In [ ]:
np.log(df.time_diff.value_counts()).plot.hist(bins = 50)

In [ ]:
#meow moew

In [ ]:
target = pd.read_csv("past_vols.csv",index_col="Unnamed: 0")

df.head()

df_size = df['size']



gb = df.groupby(by=['stock_id', 'time_id']).agg({"size":"sum", "spread": "mean","time_diff":"max"})

target = target.merge(gb.reset_index())

target['sqrt_size'] = np.sqrt(target['size'])
target['timef'] = np.log(target['time_diff'])

In [ ]:
np.log(df.spread).plot.hist(bins = 50)

In [ ]:
pd.Series(np.log(df['size'])).plot.hist(bins = 50)

In [ ]:
target.plot.scatter(x="spread", y="sigma",c= "stock_id")
#spread very high corr with vol. unsuprising

In [ ]:
target.plot.scatter(x="sqrt_size", y="sigma",c= "stock_id")

#Size low corr with sigma: prob due to low corr size ~ volume

In [ ]:
target.plot.scatter(x= "timef", y = "sigma", c="stock_id")

#Hmm very liquid mala sigma, illiquid tez, mid liquid duze zmiany?

In [ ]:
target.corr().sigma
#cool

In [ ]:
target.to_parquet("past_vols2.csv")
df.to_parquet("clean_trades.parquet")